In [ ]:
import ipywidgets
import PIL
import bqplot.pyplot

import keras
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [ ]:
def adding_problem_generator(N, seq_len=6, high=1):
    """ A data generator for the adding problem.

    The data definition strictly follows Quoc V. Le, Navdeep Jaitly, Geoffrey E.
    Hintan's paper, A Simple Way to Initialize Recurrent Networks of Rectified
    Linear Units.

    The single datum entry is a 2D vector with two rows with same length.
    The first row is a list of random data; the second row is a list of binary
    mask with all ones, except two positions sampled by uniform distribution.
    The corresponding label entry is the sum of the masked data. For
    example:

     input          label
     -----          -----
    1 4 5 3  ----->   9 (4 + 5)
    0 1 1 0

    :param N: the number of the entries.
    :param seq_len: the length of a single sequence.
    :param p: the probability of 1 in generated mask
    :param high: the random data is sampled from a [0, high] uniform distribution.
    :return: (X, Y), X the data, Y the label.
    """    
    X_num = np.random.uniform(low=0, high=high, size=(N, seq_len, 1))
    X_mask = np.zeros((N, seq_len, 1))
    Y = np.ones((N, 1))
    for i in range(N):
        # Default uniform distribution on position sampling
        positions = np.random.choice(seq_len, size=2, replace=False)
        X_mask[i, positions] = 1
        Y[i, 0] = np.sum(X_num[i, positions])
    X = np.append(X_num, X_mask, axis=2)
    return X, Y

In [ ]:
def init_plot(baseline):
    axes = {'x': {'label': 'Epochs'}, 
            'y': {'label': 'Losses', 
                  'label_offset': '50px',
                  'tick_style': {'font-size': 10}}}
    y_sc = bqplot.LinearScale(min=0.0, max=0.25)
    
    loss_plt = bqplot.pyplot.figure(min_aspect_ratio=8/5, max_aspect_ratio=8/5, scales={'y': y_sc})
    bqplot.pyplot.plot([0,1],[0.0,0.0], axes_options=axes)
    bqplot.pyplot.plot([0,1],[0.0,0.0], colors=['orange'])
    bqplot.pyplot.hline(baseline, line_style='dashed', stroke_width=2.0, colors=['red'])
    bqplot.pyplot.hline(0.0, line_style='dashed', stroke_width=2.0, colors=['green'])
    bqplot.pyplot.scatter([0], [0.0], colors=['red'], stroke_width = 0.5, marker='circle', visible=False)
    return loss_plt

class plot_history(keras.callbacks.Callback):
    def __init__(self, loss_plt):
        self.loss_plt = loss_plt
        self.history = {'loss':[], 'val_loss':[]}
    
    def on_epoch_end(self, epoch, logs={}):
        self.history['loss'].append(logs.get('loss'))
        self.history['val_loss'].append(logs.get('val_loss'))
        
        x_data = range(1, len(self.history['loss'])+1)
        self.loss_plt.marks[0].x = x_data
        self.loss_plt.marks[0].y = self.history['loss']
        self.loss_plt.marks[1].x = x_data
        self.loss_plt.marks[1].y = self.history['val_loss']

def plot_test_loss(loss_plt, epochs, loss):
    loss_plt.marks[4].x = [epochs]
    loss_plt.marks[4].y = [loss]
    loss_plt.marks[4].visible=True

In [ ]:
seqlen = 64
epochs = 300
batch_size = 100
x_train, y_train = adding_problem_generator(1000, seq_len=seqlen)

model = keras.models.Sequential()
model.add(keras.layers.GRU(8, input_shape=(seqlen, 2)))
model.add(keras.layers.Dense(1, activation='linear'))
model.compile(optimizer='rmsprop', loss='mse')
model.summary()

loss_plt = init_plot(0.1767)
display(loss_plt)

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0,
                    validation_split=0.2, callbacks=[plot_history(loss_plt)])

In [ ]:
x_test, y_test = adding_problem_generator(500, seq_len=seqlen)
test_loss = model.evaluate(x_test, y_test)
plot_test_loss(loss_plt, epochs, test_loss)